# [백준/Smallest Enclosing Sphere](https://www.acmicpc.net/problem/11930)


## 풀이과정


### 첫번째 시도


#### 풀이과정

먼저 3차원 벡터와 3x3 행렬 간 연산 시에 필요한 함수들을 정의했다. 그리고 크래머 공식을 이용해 삼차연립방정식을 푸는 `cramers` 함수를 정의했다. 이를 이용해 점이 3개가 주어졌을 때는 $\begin{pmatrix}2(b - a)\\2(c - a)\\(b-a)\times(c-a)\end{pmatrix}\cdot x = \begin{pmatrix}(b - a)^2\\(c - a)^2\\0\end{pmatrix}$ 를, 점이 4개가 주어졌을 때는 $2\begin{pmatrix}b - a \\ c - a \\ d - a\end{pmatrix}\cdot x = \begin{pmatrix}(b - a)^2\\(c - a)^2\\(d - a)^2\end{pmatrix}$ 를 푼 뒤 $a$ 를 더해 구의 중심을 구했다. 마지막으로 웰츠 알고리즘을 이용해 모든 점을 포함하는 최소 구의 반지름을 구했다.

상기한 공식은 다음과 같은 과정을 통해 유도했다.
먼저 점이 4개 주어진 경우를 생각해보자.
점 $a, b, c, d$ 가 주어졌을 때 외심의 중심을 $x$ 라고 하자.

$$
\begin{cases}
(a - x)^2 = r^2 \\
(b - x)^2 = r^2 \\
(c - x)^2 = r^2 \\
(d - x)^2 = r^2
\end{cases}
\Rightarrow \begin{cases}
(a - x)^2 = (b - x)^2  \\
(a - x)^2 = (c - x)^2  \\
(a - x)^2 = (d - x)^2
\end{cases}
\Rightarrow \begin{cases}
2(b - a)\cdot x = b^2 - a^2 \\
2(c - a)\cdot x = c^2 - a^2 \\
2(d - a)\cdot x = d^2 - a^2
\end{cases}
$$

로 풀 수 있다. 여기서 코드를 좀더 간단하게 하기 위해 $x' + a = x$ 로 치환하면 다음과 같이 표현할 수 있다. 그러면

$$
2(b - a)\cdot x = b^2 - a^2  \\
\Rightarrow 2(b - a)\cdot (x' + a) = b^2 - a^2  \\
\Rightarrow 2(b - a)\cdot x' = b^2 - a^2 - 2a \cdot (b - a) \\
\Rightarrow 2(b - a)\cdot x' = b^2 - 2a \cdot b + a^2 \\
\Rightarrow 2(b - a)\cdot x' = (b - a)^2 \\
$$

와 같이 쓸 수 있다. 따라서

$$
\begin{cases}
2(b - a)\cdot (x + a) = b^2 - a^2 \\
2(c - a)\cdot (x + a) = c^2 - a^2 \\
2(d - a)\cdot (x + a) = d^2 - a^2
\end{cases}
\Rightarrow
\begin{cases}
2(b - a)\cdot x = (b - a)^2 \\
2(c - a)\cdot x = (c - a)^2 \\
2(d - a)\cdot x = (d - a)^2
\end{cases}
\Rightarrow
2\begin{pmatrix}
b - a \\
c - a \\
d - a
\end{pmatrix} \cdot x
= \begin{pmatrix}
(b - a)^2 \\
(c - a)^2 \\
(d - a)^2
\end{pmatrix}
$$

이다. 점이 3개 주어진 경우는 점 $d$ 대신 $a, b, c, x$ 가 한 평면에 있다는 점을 이용하면 된다. $(b - a)\times(c - a)$ 는 네 점이 있는 평면에 수직이고 $x$ 는 이 평면 위에 있으므로 $(b - a)\times(c - a)\cdot x = 0$ 이다. 따라서 점이 4개인 경우의 식에서 좌변의 마지막 행을 $(b - a)\times(c - a)\cdot x$, 우변의 마지막 행을 $0$으로 바꾸면 된다.


In [ ]:
from math import hypot
from operator import __add__, __sub__, __mul__, truediv

Point = tuple[float, float, float]
Mat3 = tuple[Point, Point, Point]
Sphere = tuple[Point, float]


def det2(ab: tuple[tuple[float, float], tuple[float, float]]):
    (ax, ay), (bx, by) = ab
    return ax * by - ay * bx


def add(a: Point, b: Point) -> Point:
    return tuple(i + j for i, j in zip(a, b))


def mul(a: float, b: Point) -> Point:
    return tuple(map(lambda x: x * a, b))


def dot(a: Point, b: Point):
    return sum(map(__mul__, a, b))


def cross(a: Point, b: Point) -> Point:
    return tuple(map(det2, ((a[1:], b[1:]), (a[::-2], b[::-2]), (a[:2], b[:2]))))


def sub(a: Point, b: Point) -> Point:
    return tuple(map(__sub__, a, b))


def div(a: Point, b: float) -> Point:
    return tuple(map(lambda x: truediv(x, b), a))


def pow2(a: Point) -> float:
    return dot(a, a)


def det3(abc: Mat3):
    a, b, c = abc
    return dot(a, cross(b, c))


def t(a: Mat3) -> Mat3:
    p, q, r = a
    return tuple(zip(p, q, r))


def adjoints(a: Mat3, b: Point) -> tuple[Mat3, Mat3, Mat3]:
    x, y, z = a
    return (t((b, y, z)), t((x, b, z)), t((x, y, b)))


def cramers(a: Mat3, b: Point) -> Point:
    return tuple(det3(ad) / det3(a) for ad in adjoints(t(a), b))


def from1(a: Point = (0, 0, 0)) -> Point:
    return a


def from2(a: Point, b: Point) -> Point:
    # hypot(*sub(a, b)) / 2
    return div(add(a, b), 2)


def from3(a: Point, b: Point, c: Point) -> Point:
    ab = sub(b, a)
    ac = sub(c, a)
    A = (mul(2, ab), mul(2, ac), cross(ab, ac))
    d = (dot(ab, ab), dot(ac, ac), 0)
    return add(a, cramers(A, d))


def from4(a: Point, b: Point, c: Point, d: Point) -> Point:
    subs = tuple(sub(p, a) for p in (b, c, d))
    A = tuple(mul(2, v) for v in subs)
    d = tuple(map(pow2, subs))
    return add(a, cramers(A, d))


def get_radius(center: Point, points: list[Point]) -> float:
    return hypot(*sub(center, points[0]))


def get_circle(points: list[Point]) -> Sphere:
    center = [from2, from3, from4][len(points) - 2](*points)
    return center, get_radius(center, points)


def init(bound: list[Point]) -> Sphere:
    if len(bound) > 1:
        return get_circle(bound)
    else:
        return ((bound[0] if bound else (0, 0, 0)), 0.0)


def enclosed(bound: list[Point], points: list[Point]) -> tuple[Point, float]:
    if len(bound) == 4:
        return get_circle(bound)
    center, radius = init(bound)
    for i, p in enumerate(points):
        if hypot(*sub(center, p)) > radius:
            center, radius = enclosed(bound + [p], points[:i])
    return center, radius


def solution():
    import sys

    inputs = sys.stdin.read().strip().split("\n")[1:]
    points = list(set(tuple(map(float, line.split())) for line in inputs))
    _, radius = enclosed([], points)
    print(f"{radius:.2f}")


solution()

## 해답


In [ ]:
from math import hypot
from operator import __add__, __sub__, __mul__, truediv

Point = tuple[float, float, float]
Mat3 = tuple[Point, Point, Point]
Sphere = tuple[Point, float]

In [ ]:
def add(a: Point, b: Point) -> Point:
    return tuple(i + j for i, j in zip(a, b))


def sub(a: Point, b: Point) -> Point:
    return tuple(map(__sub__, a, b))


def mul(a: float, b: Point) -> Point:
    return tuple(map(lambda x: x * a, b))


def dot(a: Point, b: Point):
    return sum(map(__mul__, a, b))


def det2(ab: tuple[tuple[float, float], tuple[float, float]]):
    (ax, ay), (bx, by) = ab
    return ax * by - ay * bx


def cross(a: Point, b: Point) -> Point:
    return tuple(map(det2, ((a[1:], b[1:]), (a[::-2], b[::-2]), (a[:2], b[:2]))))


def div(a: Point, b: float) -> Point:
    return tuple(map(lambda x: truediv(x, b), a))


def pow2(a: Point) -> float:
    return dot(a, a)

In [ ]:
def det3(abc: Mat3):
    a, b, c = abc
    return dot(a, cross(b, c))


def t(a: Mat3) -> Mat3:
    return tuple(zip(*a))


def adjoints(a: Mat3, b: Point) -> tuple[Mat3, Mat3, Mat3]:
    x, y, z = a
    return (t((b, y, z)), t((x, b, z)), t((x, y, b)))


def cramers(a: Mat3, b: Point) -> Point:
    return tuple(det3(ad) / det3(a) for ad in adjoints(t(a), b))

In [ ]:
def from2(a: Point, b: Point) -> Point:
    return div(add(a, b), 2)


def from3(a: Point, b: Point, c: Point) -> Point:
    ab = sub(b, a)
    ac = sub(c, a)
    A = (mul(2, ab), mul(2, ac), cross(ab, ac))
    d = (dot(ab, ab), dot(ac, ac), 0)
    return add(a, cramers(A, d))


def from4(a: Point, b: Point, c: Point, d: Point) -> Point:
    subs = tuple(sub(p, a) for p in (b, c, d))
    return add(a, cramers(tuple(mul(2, v) for v in subs), tuple(map(pow2, subs))))

In [ ]:
def get_radius(center: Point, points: list[Point]) -> float:
    return hypot(*sub(center, points[0]))


def get_circle(points: list[Point]) -> Sphere:
    center = [from2, from3, from4][len(points) - 2](*points)
    return center, get_radius(center, points)

In [ ]:
def init(bound: list[Point]) -> Sphere:
    if len(bound) > 1:
        return get_circle(bound)
    else:
        return ((bound[0] if bound else (0, 0, 0)), 0.0)


def enclosed(bound: list[Point], points: list[Point]) -> tuple[Point, float]:
    if len(bound) == 4:
        return get_circle(bound)
    center, radius = init(bound)
    for i, p in enumerate(points):
        if hypot(*sub(center, p)) > radius:
            center, radius = enclosed(bound + [p], points[:i])
    return center, radius

In [ ]:
def solution():
    import sys

    inputs = sys.stdin.read().strip().split("\n")[1:]
    points = list(set(tuple(map(float, line.split())) for line in inputs))
    _, radius = enclosed([], points)
    print(f"{radius:.2f}")

## 예제


In [49]:
# 백준 문제 풀이용 예제 실행 코드
from bwj import test

test_solution = test(solution)

# test_solution("""""")
# test_solution(read("fn").read())

In [54]:
test_solution(
    """3
-747 -497 354
923 -364 -644
497 661 -702
"""
)  # 1030.21

1030.21


In [53]:
test_solution(
    """3
491 834 -78
400 503 310
-319 484 124
"""
)  # 452.60

452.60


In [ ]:
test_solution(
    """5
5 0 0
-5 0 0
0 3 4
4 -3 0
2 2 -2
"""
)  # 5.00

In [55]:
from random import randrange

MAX = 1000000
LEN = 1000

rand = lambda: randrange(-MAX, MAX)

example = f"{LEN}\n" + "\n".join(f"{rand()} {rand()} {rand()}" for _ in range(LEN))
test_solution(example)
# open("11930.in", "w").write(example)

1689136.32
